In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
cd MyDrive

In [ ]:
cd cdQA-bert_qa_vGPU

In [ ]:
pip install -e .

In [ ]:
pip install cdqa

In [ ]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline.cdqa_sklearn import QAPipeline

In [ ]:
df = pd.read_csv('./data/korean.csv', encoding='cp949',converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df['content'] = df['paragraphs'].apply(lambda x: ' '.join(x))

In [ ]:
df.head()

In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa_korquad_vCPU.joblib')
cdqa_pipeline.fit(df)
cdqa_pipeline.reader.output_dir = './logs/'

In [ ]:
query = '컴퓨터공학부 전화번호 뭐야?'
prediction = cdqa_pipeline.predict(X=query)

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))

In [ ]:
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

# Retriever 향상을 위한 pos_tagging

In [ ]:
import pandas as pd
from ast import literal_eval
import urllib3
import json
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline.cdqa_sklearn import QAPipeline
import time
from cdqa.retriever import tfidf_sklearn as tfidf

In [ ]:
def ETRI_POS_Tagging(text) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = "2188bb67-6f40-41e9-95a2-b392c2518964"
    analysisCode = "wsd"
    requestJson = {
        "access_key":accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    print("[responseCode] " + str(response.status))
    print("[responBody]")
    print(str(response.data,"utf-8"))
    return Pos_extract(response)


def Pos_extract(Data) :
    Noun = []
    print(json.loads(str(Data.data,"utf-8")))
    Extract_a = json.loads(str(Data.data,"utf-8"))['return_object']['sentence']
    for i in range(len(Extract_a)) :
        Extract_b = dict(Extract_a[i])
        for j in range(len(Extract_b['WSD'])) :
            if (Extract_b['WSD'][j]['type'] =='NNG' or Extract_b['WSD'][j]['type'] =='NNP') or Extract_b['WSD'][j]['type'] =='VV':
                Noun.append(Extract_b['WSD'][j]['text'])
    return " ".join(Noun)

In [ ]:
df = pd.read_csv('./data/korean.csv',encoding='cp949',converters={'paragraphs': literal_eval})
df['content'] = df['paragraphs'].apply(lambda x: ' '.join(x))

#유사도를 구할 retriever 선언 및 훈련
retriever = tfidf.TfidfRetriever(ngram_range=(1, 2),max_df=0.85, stop_words=None)
retriever_temp = tfidf.TfidfRetriever(ngram_range=(1, 2),max_df=0.85, stop_words=None)

#df의 content열을 선택해서 훈련
retriever.fit(df['content'])

df = filter_paragraphs(df, min_length=10, max_length=100)

In [ ]:
# 학습된 bert모델 불러오기
cdqa_pipeline = QAPipeline(reader='models/bert_qa_korquad_vCPU.joblib')

In [ ]:
best_idx_scores = ''
query = '컴퓨터공학부가 어디 건물에 있어?'

# 정책과 질문의 유사도가 1보다 높으면 해당 정책의 문서를 선택하여 재학습
# 질문의 유사도가 낮으면 해당 정책에 대해 질의응답
POS_query = ETRI_POS_Tagging(query)

# 정책과 질문의 유사도가 1보다 높으면 해당 정책의 문서를 선택하여 재학습
# 질문의 유사도가 낮으면 해당 정책에 대해 질의응답
POS_query = ETRI_POS_Tagging(query)
result = retriever.predict(POS_query, df)

if result[0] >= 4. or not best_idx_scores:
    # 질문과 유사도가 높은 정책의 인덱스 예측
    best_idx_scores = retriever.predict(POS_query, df)
    # 청년정책에서 선택된 문서 학습
    best_idx_scores_list = list(map(int, best_idx_scores))
    best_idx_scores_list = [idx for idx in best_idx_scores_list if idx < len(df)]
    print(df.loc[best_idx_scores_list].head(1))
    # 청년정책에서 선택된 문서 학습
    cdqa_pipeline.fit(df.loc[best_idx_scores_list])

# 정답 예측
prediction = cdqa_pipeline.predict(query)
print('paragraph: {}\n'.format(prediction[0]))
print('paragraph: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))